In [ ]:
# Les prérequis usuels
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


L'un des principaux problèmes que posent les données désordonnées est le suivant : comment savoir si elles sont désordonnées ou non ?

Nous allons utiliser à nouveau l'ensemble de données de rejets de reprises ici, car il est gros et un peu lourd.

In [ ]:
kapi_rejets = pd.read_csv('../data/base_kapi_rejets.csv', sep=';', encoding='ANSI')

# 4.1 Comment savoir si les données sont confuses ? 

Nous allons regarder quelques colonnes ici. Je sais déjà qu'il y a des problèmes avec le code information, alors regardons d'abord cela.
 
Pour savoir si une colonne a des problèmes, j'utilise généralement `.unique()` pour examiner toutes ses valeurs. Si c'est une colonne numérique, je vais plutôt tracer un histogramme pour avoir une idée de la distribution.

Quand on regarde les valeurs uniques de "INFO", on s'aperçoit rapidement qu'il s'agit d'un gâchis.

Quelques-uns des problèmes :

* Certains ont été analysés comme des chaînes de caractères et d'autres comme des flotteurs
* Il y a des `nan`
* Il y a des valeurs N/A que les pandas ne reconnaissaient pas, comme'N/A' et''.

Ce que nous pouvons faire :

* Normaliser 'N/A' et 'NO CLUE' en valeurs nan régulières.
* Regardez ce qui se passe avec le 83, et décidez de ce qu'il faut faire
* Faire que tout soit lié

In [ ]:
kapi_rejets['INFO'].unique()

# 7.2 *Réparer* la confusion entre les valeurs nan et les chaines/float 

Nous pouvons passer une option `na_values` à `pd.read_csv` pour nettoyer un peu cela. Nous pouvons aussi spécifier que le type de Zip d'Incident est une chaîne de caractères et non un float.

In [ ]:
na_values = ['  ', 'N/A', '0']
kapi_rejets = pd.read_csv('../data/base_kapi_rejets.csv', na_values=na_values, dtype={'INFO': str}, sep=';', encoding='ANSI')

In [ ]:
kapi_rejets['INFO'].unique()

# 7.3 C'est quoi ces blancs ?

In [ ]:
colonne_avec_blanc = kapi_rejets['INFO'].str.contains(' ').fillna(False)
len(kapi_rejets[colonne_avec_blanc])

In [ ]:
kapi_rejets[colonne_avec_blanc]

Je pensais qu'il s'agissait de données manquantes et je les ai effacées comme ceci :

`kapi_rejets['INFO'][colonne_avec_blanc] = np.nan`

Mais mon ami Marc m'a fait remarquer que les codes ave blanc sont normaux. Regardons tous les codes avec blanc, vérifions qu'ils sont corrects, puis tronquez-les.

In [ ]:
code_avec_blanc = kapi_rejets['INFO'].str.len() > 4
kapi_rejets['INFO'][code_avec_blanc].unique()

Ils ont tous l'air bons pour être tronqués.

In [ ]:
kapi_rejets['INFO'] = kapi_rejets['INFO'].str.slice(0, 4)

Fait.

Plus tôt, je pensais que ZY était un code incomplet, cependant : jetons un coup d'oeil à cela. 

In [ ]:
kapi_rejets[kapi_rejets['INFO'] == 'ZY']

Ça me semble normal..

Super. Voyons où nous en sommes maintenant :

In [ ]:
code_unique = kapi_rejets['INFO'].unique()
code_unique.sort()
code_unique

Incroyable ! C'est beaucoup plus propre. 


# 7.4 Mettre tout ensemble

Voici ce que nous avons fini par faire pour nettoyer nos codes, tous ensemble :

In [ ]:
valeur_a_exclure = ['  ', 'N/A', '0']
kapi_rejets = pd.read_csv('../data/base_kapi_rejets.csv', 
                          na_values=valeur_a_exclure, 
                          dtype={'INFO': str}, 
                          sep=';', encoding='ANSI')


In [ ]:
def fix_codes_infos(infos):
    # tronquer tout à 4 caractères 
    infos = infos.str.slice(0, 4)
    
    # Set 0000 codes to nan
    zero_infos = infos == '0000'
    infos[zero_infos] = np.nan
    
    return infos

In [ ]:
kapi_rejets['INFO'] = fix_codes_infos(kapi_rejets['INFO'])

In [ ]:
kapi_rejets['INFO'].unique()

<style>
    @font-face {
        font-family: "Computer Modern";
        src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');
    }
    div.cell{
        width:800px;
        margin-left:16% !important;
        margin-right:auto;
    }
    h1 {
        font-family: Helvetica, serif;
    }
    h4{
        margin-top:12px;
        margin-bottom: 3px;
       }
    div.text_cell_render{
        font-family: Computer Modern, "Helvetica Neue", Arial, Helvetica, Geneva, sans-serif;
        line-height: 145%;
        font-size: 130%;
        width:800px;
        margin-left:auto;
        margin-right:auto;
    }
    .CodeMirror{
            font-family: "Source Code Pro", source-code-pro,Consolas, monospace;
    }
    .text_cell_render h5 {
        font-weight: 300;
        font-size: 22pt;
        color: #4057A1;
        font-style: italic;
        margin-bottom: .5em;
        margin-top: 0.5em;
        display: block;
    }
    
    .warning{
        color: rgb( 240, 20, 20 )
        }  